In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F

In [4]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data

In [5]:
# find gpu
cuda = torch.cuda.is_available()

In [6]:
# hyperparameters
NUM_LEVELS = 4
tau = 1.
SEED = 1

In [7]:
torch.manual_seed(SEED)
if cuda:
    torch.cuda.manual_seed(SEED)

In [8]:
#transform
data_transforms = transforms.Compose([
        transforms.ToTensor(),
        lambda x: x>0.5,
        lambda x: x.float(),
    ])

In [9]:
# dataset loading
bsize = 100
train_loader = torch.utils.data.DataLoader(
    dset.MNIST('data', train=True, download=True, transform=data_transforms), batch_size=bsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    dset.MNIST('data', train=False, download=True, transform=data_transforms), batch_size=bsize, shuffle=True)

In [10]:
def sample_binary_concrete(inputs, temperature = tau):
    U = Variable(torch.rand(inputs.shape), requires_grad=False)
    if cuda:
        U = U.cuda()
    return F.sigmoid((U.log() - (1-U).log() + inputs)/temperature)

def hard_sample_binary_concrete(inputs):
    y = sample_binary_concrete(inputs)
    y_hard = torch.round(y)
    return (y_hard - y).detach() + y

In [11]:
class Encoder(nn.Module):
    def __init__(self, num_levels=NUM_LEVELS):
        super(Encoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
#         self.fc = nn.Sequential(
#             nn.Linear(784, 256),
#             nn.ReLU(),
#             nn.Linear(256, 64),
#             nn.ReLU(),
#         )
        
        # TODO try to make tunnels smaller as we go higher.
#         self.tunnels = nn.ModuleList([nn.Linear(128,128) for _ in range(num_levels-1)])
        # XXX temp try
        self.tunnels = nn.ModuleList([
            nn.Linear(128,64),
            nn.Linear(64,32),
            nn.Linear(32,16),
        ])
#         self.tunnels = nn.ModuleList([
#             nn.Linear(64,32),
#             nn.Linear(32,16),
#             nn.Linear(16,8),
#         ])
        
        fc_output_shape = self.fc[-2].weight.shape[0]
        coding_inp_sizes = [fc_output_shape]
        for i in range(len(self.tunnels)):
            coding_inp_sizes.append(self.tunnels[i].weight.shape[0])
        
        self.codings = nn.ModuleList([
            nn.Linear(
                coding_inp_sizes[_], 2**(num_levels-_-1) 
            ) for _ in range(num_levels)])  
        print(self.tunnels)
        print(self.codings)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        sofar = self.fc(x)
        posteriors = [self.codings[0](sofar)]
        for _ in range(len(self.tunnels)):
            sofar = self.relu(self.tunnels[_](sofar))
            posteriors.append(self.codings[_+1](sofar))
        return posteriors
    
e = Encoder()
_, (data, _) = next(enumerate(train_loader))
data = data.view(-1, 784)
data = Variable(data, requires_grad=False)
if cuda:
    data = data.cuda()
    e.cuda()
post = e(data)
print(len(post))
print(post[0].shape)
print(post[1].shape)
print(post[2].shape)
print(post[3].shape)

ModuleList(
  (0): Linear(in_features=128, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=32, bias=True)
  (2): Linear(in_features=32, out_features=16, bias=True)
)
ModuleList(
  (0): Linear(in_features=128, out_features=8, bias=True)
  (1): Linear(in_features=64, out_features=4, bias=True)
  (2): Linear(in_features=32, out_features=2, bias=True)
  (3): Linear(in_features=16, out_features=1, bias=True)
)
4
torch.Size([100, 8])
torch.Size([100, 4])
torch.Size([100, 2])
torch.Size([100, 1])


In [12]:
class Decoder(nn.Module):
    def __init__(self, num_levels=NUM_LEVELS):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(2**(num_levels-1), 64)
        self.fc2 = nn.Linear(64,256)
        self.fc3 = nn.Linear(256,784)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, code):
        h = self.relu(self.fc1(code))
        h = self.relu(self.fc2(h))
        y = self.sigmoid(self.fc3(h))
        return y
    
d = Decoder()
if cuda:
    d.cuda()
d(post[0]).shape

torch.Size([100, 784])

In [13]:
class Hierarchy(nn.Module):
    def __init__(self, num_levels=NUM_LEVELS):
        super(Hierarchy, self).__init__()
        self.root_dist = nn.Parameter(torch.Tensor([0.5]))
        self.downwards = nn.ModuleList([nn.Linear(2**(_-1),2**_) for _ in range(num_levels-1, 0, -1)])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, codes):
        n_codes = len(codes)
        cond_priors = [self.sigmoid(self.root_dist.expand(codes[n_codes-1].size()[0],1))]
        # codes are 8, 4, 2, 1, so are downwards
        for i in range(n_codes-1, 0, -1):
            cond_priors.append(self.sigmoid(self.downwards[i-1](codes[i])))
        return cond_priors

h = Hierarchy()
if h:
    h.cuda()
print(h.downwards)
codes = [hard_sample_binary_concrete(posterior) for posterior in post]
prior = h(codes)
print(len(prior))
print(prior[0].shape)
print(prior[1].shape)
print(prior[2].shape)
print(prior[3].shape)

ModuleList(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): Linear(in_features=2, out_features=4, bias=True)
  (2): Linear(in_features=1, out_features=2, bias=True)
)
4
torch.Size([100, 1])
torch.Size([100, 2])
torch.Size([100, 4])
torch.Size([100, 8])


In [14]:
class Model(nn.Module):
    def __init__(self, num_levels=NUM_LEVELS):
        super(Model, self).__init__()
        self.E = Encoder(num_levels)
        self.D = Decoder(num_levels)
        self.H = Hierarchy(num_levels)
        
    def forward(self, x):
        posteriors = self.E(x)
        codes = [hard_sample_binary_concrete(posterior) for posterior in posteriors]
        cond_priors = self.H(codes)
        y = self.D(codes[0])
        cond_priors.reverse() # to make it the same shape as posteriors
        return cond_priors, posteriors, y
        
    def generate(self):
        # TODO
        pass
    
    def save(self):
        torch.save(self.state_dict(), 'generative_mnist_U_shaped.save')
        
    def load(self):
        self.load_state_dict(torch.load('generative_mnist_U_shaped.save'))
    
m = Model()
_, (data, _) = next(enumerate(train_loader))
data = data.view(-1, 784)
data = Variable(data, requires_grad=False)
if cuda:
    m.cuda()
    data = data.cuda()
cond_priors, posteriors, y = m(data)
# assert they're of the same shape order, to prevent stupid mistake in KLD
assert cond_priors[0].shape == posteriors[0].shape

ModuleList(
  (0): Linear(in_features=128, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=32, bias=True)
  (2): Linear(in_features=32, out_features=16, bias=True)
)
ModuleList(
  (0): Linear(in_features=128, out_features=8, bias=True)
  (1): Linear(in_features=64, out_features=4, bias=True)
  (2): Linear(in_features=32, out_features=2, bias=True)
  (3): Linear(in_features=16, out_features=1, bias=True)
)


In [15]:
def KLD(cond_priors, posteriors):
    cond_priors = torch.cat(cond_priors, dim = -1)
    posteriors = torch.cat(posteriors, dim = -1) 
    posteriors = F.sigmoid(posteriors)
    const = 1E-20
    kld_per_latent = posteriors * ((posteriors+const).log() - (cond_priors+const).log()) + \
                    (1-posteriors) * ((1-posteriors+const).log() - (1-cond_priors+const).log())
    return kld_per_latent.sum(dim=-1).mean()

def recon_loss(y, true_y):
    return F.binary_cross_entropy(y, true_y, size_average=False)/y.size()[0]

In [16]:
model = Model()
if cuda:
    model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=1e-3)
best_loss = 9999999

ModuleList(
  (0): Linear(in_features=128, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=32, bias=True)
  (2): Linear(in_features=32, out_features=16, bias=True)
)
ModuleList(
  (0): Linear(in_features=128, out_features=8, bias=True)
  (1): Linear(in_features=64, out_features=4, bias=True)
  (2): Linear(in_features=32, out_features=2, bias=True)
  (3): Linear(in_features=16, out_features=1, bias=True)
)


In [17]:
def train(epoch):
    global best_loss
    train_loss = 0
    kld_loss = 0
    rec_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        data = Variable(data, requires_grad=False)
        if cuda:
            data = data.cuda()
        x = data.view(-1,784)
        
        cond_priors, posteriors, y = model(x)
        
        kld = KLD(cond_priors, posteriors)
        rec = recon_loss(y, x)
        loss = kld + rec
        
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0]
        kld_loss += kld.data[0]
        rec_loss += rec.data[0]
        
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tKLD: {:.6f}\trecon_loss:{:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                batch_idx*100/ len(train_loader),
                loss.data[0], kld.data[0], rec.data[0]))
    print('====> Epoch: {} Average loss: {:.4f}\tKLD: {:.6f}\trecon_loss:{:.6f}'.format(
          epoch+1, train_loss / len(train_loader), kld_loss/ len(train_loader), rec_loss/ len(train_loader)))
    
    cur_loss = train_loss / len(train_loader)
    if cur_loss < best_loss:
        model.save()
        best_loss = cur_loss

In [ ]:
%%time
train(0)

Train Epoch: 1 [0/60000 (0%)]	Loss: 546.292236	KLD: 0.525637	recon_loss:545.766602
Train Epoch: 1 [10000/60000 (17%)]	Loss: 210.676529	KLD: 4.039052	recon_loss:206.637482
Train Epoch: 1 [20000/60000 (33%)]	Loss: 208.550690	KLD: 2.499348	recon_loss:206.051346
Train Epoch: 1 [30000/60000 (50%)]	Loss: 210.839233	KLD: 1.555880	recon_loss:209.283356


In [ ]:
# training
epochs = 25
for epoch in range(1,epochs):
    train(epoch)

In [ ]:
print("Loading best model with loss {}".format(best_loss))
model.load()

# Show everything that could be recovered.

In [ ]:
from itertools import product

def debug(start=0, end=32):
    NN = 2**(NUM_LEVELS-1) # `end` is up to NN
    f, axes = plt.subplots(1, end-start, sharey=True, figsize=(25,1))
    k = 0
    kk = 0

    def make_variable(value):
        t = torch.Tensor([value]).type(torch.FloatTensor)
        t = t.unsqueeze(0)
        v = Variable(t)
        if cuda:
            v = v.cuda()
        return v

    for path in product(range(2), repeat=NN):
        if k >= start and k < end:
            decoder_hidden = make_variable(path)
            w = model.D(decoder_hidden)
            ax = axes[kk]
            ax.axis('off')
            #ax.set_title(path)
            ax.imshow(w.view(28,28).cpu().data.numpy())
            kk += 1
        k += 1
    plt.show()
    
for i in range(int(2**(2**(NUM_LEVELS-1))/32)):
    debug(start=i*32, end=(i+1)*32)

In [ ]:
def generate_image():
    sofar = hard_sample_binary_concrete(model.H.root_dist)
    #print(sofar)
    for i in range(len(model.H.downwards)-1, -1, -1):
        sofar = model.H.downwards[i](sofar)
        sofar = hard_sample_binary_concrete(sofar)
        #print(sofar)
    generated = model.D(sofar)
    return generated

generated = generate_image()
generated_img = generated.view(28,28).data.cpu()
plt.imshow(generated_img)

In [ ]:
def generate_image_fixed_first_level(value):
    sofar = value#hard_sample_binary_concrete(model.H.root_dist)
    #print(sofar)
    for i in range(len(model.H.downwards)-1, -1, -1):
        sofar = model.H.downwards[i](sofar)
        sofar = hard_sample_binary_concrete(sofar)
        #print(sofar)
    generated = model.D(sofar)
    return generated

f, axes = plt.subplots(1, 32, sharey=True, figsize=(25,1))
for i in range(32):
    value = Variable(torch.Tensor([0]))
    if cuda:
        value = value.cuda()
    generated = generate_image_fixed_first_level(value)
    generated_img = generated.view(28,28).data.cpu()
    ax = axes[i]
    ax.imshow(generated_img)
plt.show()

f, axes = plt.subplots(1, 32, sharey=True, figsize=(25,1))
for i in range(32):
    value = Variable(torch.Tensor([1]))
    if cuda:
        value = value.cuda()
    generated = generate_image_fixed_first_level(value)
    generated_img = generated.view(28,28).data.cpu()
    ax = axes[i]
    ax.imshow(generated_img)
plt.show()